In [2]:
import os
import pickle as pickle
import tensorflow as tf
import numpy as np
from Stats_original import WeightQuantization, IntroduceFaultsInWeights, GenerateFaultsList, CheckAccuracyAndLoss
from Nets import GetNeuralNetworkModel
from Training import GetDatasets
from Simulation import get_all_outputs
from Simulation import buffer_simulation, save_obj, load_obj
from funciones import compilNet, same_elements
import pandas as pd
from datetime import datetime
from pandas import ExcelWriter
from openpyxl import Workbook
from openpyxl import load_workbook
import tensorflow_datasets as tfds

In [3]:
trainBatchSize = testBatchSize = 1
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, trainBatchSize, testBatchSize)

C:\Users\usuario\anaconda3\envs\env_first\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


In [4]:
word_size  = 16
afrac_size = 11  
aint_size  = 4
wfrac_size = 11
wint_size  = 4
shift = 2**(16-1)
factor = 2**11

cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'AlexNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')

In [5]:
Cargar_errores = True



if Cargar_errores:
    locs  = load_obj('Data/Fault Characterization/error_mask y locs_buffer_act_vc-707/locs_054')
    error_mask = load_obj('Data/Fault Characterization/error_mask y locs_buffer_act_vc-707/error_mask_054')
#locs= locs[0:1]  
#error_mask=error_mask[0:1]
    
#print('mostrando las 5 primeras direcciones con fallos')
#print('direcciones:',locs)
#print('mascara de fallos:',error_mask)
print(len(error_mask))
print(len(locs))

4738
4738


In [ ]:
# tareas 
Sábado 
# visualizar el fichero riginal y ver los datos y aclarar con los tutores que el experimento lo hacemos inyectando 1 y 0 
#aleatorios
#probar lo de la invertir lo d elos bits mas significativos seleccionando del excel las posiciones de los elementos que
#puedo invertir que serian aquellos que de atras para alante tengan fallo en el 4to bit porque si es antes rotar no funciona
# de estos elementos tengo las direcciones debo machar los excel o obtener un exel que me lo de todo junto.
# Obtener el excel donde analizo los errores por distancia, posicion y fallos para el vl_54 pero con el fichero de erores a 
#o y unos y todo en un mismo excel , para así tenerlo para cada buffer.
#***Unir en un mismo .Net los 4 experimentos y enviar una variable en true que me diga cuan experimento quiero hacer
# luego de obtenido el error_mask le envio las direcciones con las palabras de 2 fallos a coregir y estas las invierto corro los 
#experimentos a ver que pasa 

#Domingo
bloc notas amarillo
#Experimento 1
#Experimento 2
#
#Lunes 
#Experimento 3

In [6]:

Net2 = GetNeuralNetworkModel('AlexNet', (227,227,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                             aging_active=True, word_size=word_size, frac_size=afrac_size, batch_size = testBatchSize)

#Cuantizacion de los pesos

loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
Net2.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
Net2.load_weights(wgt_dir).expect_partial()
WeightQuantization(model = Net2, frac_bits = 11, int_bits = 4)
#loss_sf,acc_sf =Net2.evaluate(test_dataset)
#X = [x for x,y in test_dataset]
##salidas del modelo sin fallas para la primer imagen del dataset de prueba
#out= get_all_outputs(Net2,X[0])
##out=get_all_outputs(Net2,iterator)
##salidas del modelo con fallas para la primer imagen del dataset de prueba
#out_quantizacion = out[1][0]
#out_error_and_correction = out[2][0]
#print('out_quantizacion',out_quantizacion)
#print('out_error_and_correction',out_error_and_correction)
#print('maxima diferencia absoluta entre modelo con fallas y sin fallas: ', tf.math.reduce_sum(tf.math.abs(tf.math.subtract(out_quantizacion,out_error_and_correction ))))


AlexNet
aging_active[0] True
AddCustomLayers
aging_active[1] True
AddCustomLayers
aging_active[2] True
AddCustomLayers
aging_active[3] True
AddCustomLayers
aging_active[5] True
AddCustomLayers
aging_active[6] True
AddCustomLayers
aging_active[7] True
aging_active[8] True
AddCustomLayers agin 9 True
AddCustomLayers
aging_active[9] True
aging_active[10] True
AddCustomLayers agin 10
Dense output_shape
AddCustomLayers False 1
AddCustomLayers False 2


In [ ]:
write_layer=[2,8,10,16,18,24,30,36,38,44,49,53]
out_input=[]
out_output=[]
for i,j in enumerate(write_layer):
    
    
    print('Capa',j,Net2.layers[j].__class__.__name__)
    X = [x for x,y in test_dataset]
    #salidas del modelo sin fallas para la primer imagen del dataset de prueba
    out= get_all_outputs(Net2,X[0])
    out_quantizacion = out[j-1]
    out_error_and_correction = out[j]
    diff_inp_output= tf.math.reduce_sum(tf.math.abs(tf.math.subtract(out_quantizacion,out_error_and_correction )))
    razon=tf.math.divide(tf.math.reduce_sum(tf.math.abs(out_quantizacion)),tf.math.reduce_sum(tf.math.abs(out_error_and_correction)))
    print(' diferencia absoluta input y outputs: ', tf.math.reduce_sum(tf.math.abs(tf.math.subtract(out_quantizacion,out_error_and_correction ))))
    #print('razón absoluta entre input y outputs: ', tf.math.reduce_sum(tf.math.abs(tf.math.divide (out_quantizacion,out_error_and_correction ))))
    print('prueba',tf.math.divide(tf.math.reduce_sum(tf.math.abs(out_quantizacion)),tf.math.reduce_sum(tf.math.abs(out_error_and_correction))))
    
    #print('razón absoluta entre input y outputs: ', tf.math.divide( (tf.math.reduce_sum(tf.math.abs(out_quantizacion)),tf.math.reduce_sum(tf.math.abs(out_error_and_correction) ))))

Capa 2 Lambda


In [27]:
X = [x for x,y in test_dataset]
#salidas del modelo sin fallas para la primer imagen del dataset de prueba
out= get_all_outputs(Net2,X[10])
#out=get_all_outputs(Net2,iterator)
#salidas del modelo con fallas para la primer imagen del dataset de prueba
out_quantizacion = out[1]
out_error_and_correction = out[2]
print('out_quantizacion',out_quantizacion)
print('out_error_and_correction',out_error_and_correction)
print('maxima diferencia absoluta entre modelo con fallas y sin fallas: ', tf.math.reduce_sum(tf.math.abs(tf.math.subtract(out_quantizacion,out_error_and_correction ))))
#iterator = next(iter(test_dataset))[0]
#    out=get_all_outputs(Net2,iterator)
#    out_quantizacion = out[1][0]  ## segunda capa, primer elemento del batch
#    out_error_and_correction = out[2][0]  ## tercera capa, primer elemento del batch      


dentro de active
dentro de apply round
tensor dentro de RoundValues
diferencia absoluta tf.Tensor(489.77393, shape=(), dtype=float32)
dentro de active
dentro de apply round
tensor dentro de RoundValues
diferencia absoluta tf.Tensor(1705.21, shape=(), dtype=float32)
dentro de active
dentro de apply round
tensor dentro de RoundValues
diferencia absoluta tf.Tensor(245.81738, shape=(), dtype=float32)
dentro de active
dentro de apply round
tensor dentro de RoundValues
diferencia absoluta tf.Tensor(850.8877, shape=(), dtype=float32)
dentro de active
dentro de apply round
tensor dentro de RoundValues
diferencia absoluta tf.Tensor(175.96436, shape=(), dtype=float32)
dentro de active
dentro de apply round
tensor dentro de RoundValues
diferencia absoluta tf.Tensor(229.22949, shape=(), dtype=float32)
dentro de active
dentro de apply round
tensor dentro de RoundValues
diferencia absoluta tf.Tensor(228.64502, shape=(), dtype=float32)
dentro de active
dentro de apply round
tensor dentro de RoundValu

In [11]:

for i in range(3):
    print(Net2.layers[i])

In [7]:
# Se optienen los outpust iterando solo sobre la primera imagen del dataset
## Las primeras 5 capas son: Input->Quantizacion->Error+Correccion->Convolucion->Quantizacion,
## por ende si comparamos las salidas de la capa de quantizacion con las de Error+Correccion podremos comprobar si se aplico
## la aproximacion.

In [18]:
for i in range(3):
    
    iterator = next(iter(test_dataset))[0]
    out=get_all_outputs(Net2,iterator)
    out_quantizacion = out[1][0]  ## segunda capa, primer elemento del batch
    out_error_and_correction = out[2][0]  ## tercera capa, primer elemento del batch                


dentro de active
affectedValues_array [<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.95703125, 0.95703125, 0.94921875, 0.94921875, 0.94921875],
      dtype=float32)>]
dentro de apply round
tensor entero valores afectados  tf.Tensor([1960 1960 1944 1944 1944], shape=(5,), dtype=int32)
tensor dentro de RoundValues tf.Tensor([1960 1960 1944 1944 1944], shape=(5,), dtype=int32)
tensor_with_error [2024 18344 18392 18328 18392]
tensor valores_afectados tf.Tensor([ True  True  True  True  True], shape=(5,), dtype=bool)
valores malos tf.Tensor([ 2024 18344 18392 18328 18392], shape=(5,), dtype=int32)
valores buenos tf.print tf.Tensor([1960 1960 1944 1944 1944], shape=(5,), dtype=int32)
mask del for [64 16384 16384 16384 16384]
newValues ANTES  [1920 0 0 0 0]
newValues DESPUES de colocar 0 tf.Tensor([1920    0    0    0    0], shape=(5,), dtype=int32)
tensor_act- antes de actualizar tf.Tensor([1960 1960 1944 1944 1944], shape=(5,), dtype=int32)
tensor_act que envio para inyectar errore

tensor luego de aplicar la inyección de errores última tf.Tensor([ 0.93603516  8.          8.03125    -8.         -8.03125   ], shape=(5,), dtype=float32)
newValues_array [<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([ 0.93603516,  8.        ,  8.03125   , -8.        , -8.03125   ],
      dtype=float32)>]
affectedValues [0.936035156 1.06201172 0.931152344 -0.428222656 -0.428222656]
newValues para actualizar el tensor final [0.936035156 8 8.03125 -8 -8.03125]
velores del tensor final en index_list [0.936035156 8 8.03125 -8 -8.03125]
diferencia absoluta tf.Tensor(29.21289, shape=(), dtype=float32)
dentro de active
affectedValues_array [<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([-0.44873047, -0.44873047, -0.44873047, -1.574707  , -1.574707  ],
      dtype=float32)>]
dentro de apply round
tensor entero valores afectados  tf.Tensor([33687 33687 33687 35993 35993], shape=(5,), dtype=int32)
tensor dentro de RoundValues tf.Tensor([33687 33687 33687 35993 35993], shape=(5,), dt

tensor valores_afectados tf.Tensor([ True  True  True  True  True], shape=(5,), dtype=bool)
valores malos tf.Tensor([ 2024 18344 18392 18328 18392], shape=(5,), dtype=int32)
valores buenos tf.print tf.Tensor([1960 1960 1944 1944 1944], shape=(5,), dtype=int32)
mask del for [64 16384 16384 16384 16384]
newValues ANTES  [1920 0 0 0 0]
newValues DESPUES de colocar 0 tf.Tensor([1920    0    0    0    0], shape=(5,), dtype=int32)
tensor_act- antes de actualizar tf.Tensor([1960 1960 1944 1944 1944], shape=(5,), dtype=int32)
tensor_act que envio para inyectar errores nuevamente tf.Tensor([1920    0    0    0    0], shape=(5,), dtype=int32)
new_Values que llega de redondeo tf.Tensor([1920    0    0    0    0], shape=(5,), dtype=int32)
tensor roun luego de inyectar errores entero [1984 16384 16448 16384 16448]
tensor luego de aplicar la inyección de errores última tf.Tensor([0.96875 8.      8.03125 8.      8.03125], shape=(5,), dtype=float32)
newValues_array [<tf.Tensor: shape=(5,), dtype=float

newValues para actualizar el tensor final [0.936035156 8 8.03125 -8 -8.03125]
velores del tensor final en index_list [0.936035156 8 8.03125 -8 -8.03125]
diferencia absoluta tf.Tensor(29.21289, shape=(), dtype=float32)
dentro de active
affectedValues_array [<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([-0.44873047, -0.44873047, -0.44873047, -1.574707  , -1.574707  ],
      dtype=float32)>]
dentro de apply round
tensor entero valores afectados  tf.Tensor([33687 33687 33687 35993 35993], shape=(5,), dtype=int32)
tensor dentro de RoundValues tf.Tensor([33687 33687 33687 35993 35993], shape=(5,), dtype=int32)
tensor_with_error [33751 50071 50135 52377 52441]
tensor valores_afectados tf.Tensor([ True  True  True  True  True], shape=(5,), dtype=bool)
valores malos tf.Tensor([33751 50071 50135 52377 52441], shape=(5,), dtype=int32)
valores buenos tf.print tf.Tensor([33687 33687 33687 35993 35993], shape=(5,), dtype=int32)
mask del for [64 16384 16384 16384 16384]
newValues ANTES  [33664

newValues ANTES  [1920 0 0 0 0]
newValues DESPUES de colocar 0 tf.Tensor([1920    0    0    0    0], shape=(5,), dtype=int32)
tensor_act- antes de actualizar tf.Tensor([1960 1960 1944 1944 1944], shape=(5,), dtype=int32)
tensor_act que envio para inyectar errores nuevamente tf.Tensor([1920    0    0    0    0], shape=(5,), dtype=int32)
new_Values que llega de redondeo tf.Tensor([1920    0    0    0    0], shape=(5,), dtype=int32)
tensor roun luego de inyectar errores entero [1984 16384 16448 16384 16448]
tensor luego de aplicar la inyección de errores última tf.Tensor([0.96875 8.      8.03125 8.      8.03125], shape=(5,), dtype=float32)
newValues_array [<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0.96875, 8.     , 8.03125, 8.     , 8.03125], dtype=float32)>]
affectedValues [0.95703125 0.95703125 0.94921875 0.94921875 0.94921875]
newValues para actualizar el tensor final [0.96875 8 8.03125 8 8.03125]
velores del tensor final en index_list [0.96875 8 8.03125 8 8.03125]
diferencia

newValues para actualizar el tensor final [0.936035156 8 8.03125 -8 -8.03125]
velores del tensor final en index_list [0.936035156 8 8.03125 -8 -8.03125]
diferencia absoluta tf.Tensor(29.21289, shape=(), dtype=float32)
dentro de active
affectedValues_array [<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([-0.44873047, -0.44873047, -0.44873047, -1.574707  , -1.574707  ],
      dtype=float32)>]
dentro de apply round
tensor entero valores afectados  tf.Tensor([33687 33687 33687 35993 35993], shape=(5,), dtype=int32)
tensor dentro de RoundValues tf.Tensor([33687 33687 33687 35993 35993], shape=(5,), dtype=int32)
tensor_with_error [33751 50071 50135 52377 52441]
tensor valores_afectados tf.Tensor([ True  True  True  True  True], shape=(5,), dtype=bool)
valores malos tf.Tensor([33751 50071 50135 52377 52441], shape=(5,), dtype=int32)
valores buenos tf.print tf.Tensor([33687 33687 33687 35993 35993], shape=(5,), dtype=int32)
mask del for [64 16384 16384 16384 16384]
newValues ANTES  [33664

In [10]:
iterator = next(iter(test_dataset))[0]
out=get_all_outputs(Net2,iterator)
out_quantizacion = out[1][0]  ## segunda capa, primer elemento del batch
out_error_and_correction = out[2][0]  ## tercera capa, primer elemento del batch                



dentro de active
dentro de apply round
tensor dentro de RoundValues
diferencia absoluta tf.Tensor(424.3872, shape=(), dtype=float32)
dentro de active
dentro de apply round
tensor dentro de RoundValues
diferencia absoluta tf.Tensor(1704.6719, shape=(), dtype=float32)
dentro de active
dentro de apply round
tensor dentro de RoundValues
diferencia absoluta tf.Tensor(254.77051, shape=(), dtype=float32)
dentro de active
dentro de apply round
tensor dentro de RoundValues
diferencia absoluta tf.Tensor(836.6123, shape=(), dtype=float32)
dentro de active
dentro de apply round
tensor dentro de RoundValues
diferencia absoluta tf.Tensor(175.1919, shape=(), dtype=float32)
dentro de active
dentro de apply round
tensor dentro de RoundValues
diferencia absoluta tf.Tensor(223.88184, shape=(), dtype=float32)
dentro de active
dentro de apply round
tensor dentro de RoundValues
diferencia absoluta tf.Tensor(221.80273, shape=(), dtype=float32)
dentro de active
dentro de apply round
tensor dentro de RoundValu

In [13]:
Valores_antes   = [out_quantizacion  [28] [28] [1],out_quantizacion [29] [41] [1],out_quantizacion [30] [10] [1]]
Valores_despues = [out_error_and_correction[28] [28] [1],out_error_and_correction[29] [41] [1],out_error_and_correction[30] [10] [1]]

for i in range(3):
    print('Error: ',error_mask[i])
    print('Valor antes: ',Valores_antes[i])
    print('Valor con error + aproximacion: ',Valores_despues[i])
    #print('Valor antes: ',Valores_antes[i],'En binario: ',convert(Valores_antes[i]))
    #print('Valor con error + aproximacion: ',Valores_despues[i],'En binario: ',convert(Valores_despues[i]))
    print('-------------------------')

Error:  x0xxxxxxx0xxxxxx
Valor antes:  0.95703125
Valor con error + aproximacion:  0.95703125
-------------------------
Error:  x0xxxxxxx0xxxxxx
Valor antes:  0.9609375
Valor con error + aproximacion:  0.9609375
-------------------------
Error:  x0xxxxxxx0xxxxxx
Valor antes:  0.95214844
Valor con error + aproximacion:  0.95214844
-------------------------


In [13]:
def convert(value):
    shift   = 2**(16-1)
    factor  = 2**11
    new_val = value*factor
    new_val = -new_val + shift if new_val < 0 else new_val
    new_val = int(new_val)
    return format(new_val, '#018b')

In [ ]:
index_list [[0, 28, 28, 1], [0, 29, 41, 1], [0, 30, 10, 1], [0, 31, 7, 1], [0, 32, 4, 1]]

In [ ]:
## Test para ver si los resultados son correctos
locs= locs[0:5]  
error_mask=error_mask[0:5]

In [ ]:
index_list [[0, 6, 7, 24], [0, 9, 8, 24], [0, 11, 0, 24], [0, 1, 3, 25], [0, 1, 11, 25]]
mod_list [[49087, 0], [49087, 0], [49087, 0], [49087, 0], [65471, 16384]]

In [ ]:
v_m 33202 33202 33202 33080 18048
v_b 33266 33266 33266 33144  1664
mask 64    64    64    64   16384
roun 33279 33279 33279 33151     0
     33215 33215 33215 33087 16384 luego de inyectar errores
     33215 33215 33215 33087 16384

test 1
0b1000000110110010 v_m
0b1000000111110010 v_b
  0000000001000000 mask 64
    #error a 0 vb>vm
0b0000000000111111 mask-1 63 
0b1000000111110010 v_b  or
0b1000000111111111 nuevo valor 33279


test 2
0b1000000100111000 v_m 33080
0b1000000101111000 v_b 33144
0b0000000001000000 mak  64
0b0000000000111111 mask-1 63
0b1000000101111000 v_b 33144 or
0b1000000101111111 nuevo valor   33151



In [ ]:
## Intección de errores para rectificar los resultados

In [9]:
new_Values=tf.convert_to_tensor([33279, 33279, 33279, 33151, 0])
mod_list= ([49087,0], [49087,0], [49087,0], [49087, 0], [65471,16384])
faults=tf.convert_to_tensor(mod_list)
new_Values

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([33279, 33279, 33279, 33151,     0])>

In [36]:
def inyectar_error(tensor, faults):
    tensor = tf.bitwise.bitwise_and(new_Values,faults[:,0])
    tensor = tf.bitwise.bitwise_or(tensor,faults[:,1])
    return tensor

In [ ]:
#Compruevo los valores afectados del tensor original inyectando los errores sin aplicar redeondeo 
#para ver que valores serian los erroneos en float

In [11]:
valores_afectados=tf.convert_to_tensor([-0.243164062, -0.243164062, -0.243164062 ,-0.18359375, 0.8125])
valores_afectados

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([-0.24316406, -0.24316406, -0.24316406, -0.18359375,  0.8125    ],
      dtype=float32)>

In [37]:
def inyec_convert_float(valores_afectados,faults):
    
    Ogdtype = valores_afectados.dtype
    shift   = 2**(16-1)
    factor  = 2**11
    tensor  = tf.cast(valores_afectados*factor,dtype=tf.int32)
    tensor  = tf.where(tf.less(tensor, 0), -tensor + shift , tensor )
    tensor  = tf.bitwise.bitwise_and(tensor,faults[:,0])
    tensor  = tf.bitwise.bitwise_or(tensor,faults[:,1])
    tensor  = tf.where(tf.greater_equal(tensor,shift), shift-tensor , tensor )
    tensor  = tf.cast(tensor/factor,dtype = Ogdtype)
    return tensor

In [ ]:
valores_afectados=-0.243164062 -0.243164062 -0.243164062   -0.18359375 0.8125
nuevos valores   =-0.218261719 -0.218261719 -0.218261719   -0.155761719 8
vals_afect_error= -0.21191406  -0.21191406  -0.21191406     -0.15234375  8.8125

# En este ejemplo lo que veo es que el valor original afectado en la ultima posición de la lista q
#que fue llevado a 0 en el redondeo , con la version 3 el valor queda igual al originasl 
#  cuando hago el analisis en enteros estos valores son diferentes 

In [ ]:
#Test 2
locs= locs[0:5]  
error_mask=error_mask[0:5]

In [ ]:
valores afectados: 1952 1952 1942 1952
tensor_with_error :1954 18400 18390 1952
valores_a cambiar :True  True  True False
v_m:                1954 18400 18390
v_b :               1952 1952 1942 
mask                [2 16384 16384]
newValues ANTES      [1952 0 0]
newValues DESPUES   [1952    0    0]
tensor_act         ([1952    0    0 1952] 
tensor roun luego de inyectar errores entero [1954 16448 16448 1952]                    
affectedValues                           [0.953125 0.953125 0.948242188 0.953125]
newValues para actualizar el tensor final [0.954101562 8.03125 8.03125 0.953125]    

In [ ]:
test # 1
0b11110100000 v_b
0b11110100010 v_m
0b00000000010 mask 2
0b00000000001 mask-1
0b11111111110 n_mask-1
0b11110100000 v_b and
0b11110100000 nuev_val 1952

test # 1
0b0000011110100000 v_b
0b0100011111100000 v_m
0b0100000000000000 mask 16384
0b0011111111111111 mak-1
0b1100000000000000 notmask
0b0000011110100000 v_b and
0b0000000000000000 nuev_val 0

test#3
0b0000011110010110 v_b
0b0100011111010110 v_m
0b0100000000000000 16384
0b0011111111111111 mak-1
0b1100000000000000 notmask
0b0000011110010110 v_b and
0b0000000000000000 nuev_val 0










In [19]:
bin(18390)

'0b100011111010110'

In [17]:
int('0b0000011110100000',2)

1952

In [21]:
new_Values=tf.convert_to_tensor([1952 ,   0  ,  0, 1952])
mod_list= ([65535, 514], [65535, 16448], [65535, 16448], [49087, 0])
faults=tf.convert_to_tensor(mod_list)
new_Values

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([1952,    0,    0, 1952])>

In [36]:
tensor_round=intectar_error(new_Values,faults)
tensor_round

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 1954, 16448, 16448,  1952])>

In [37]:
tensor = tf.where(tf.greater_equal(tensor_round,shift), shift-tensor_round , tensor_round )
tensor = tf.cast(tensor/factor,dtype = tf.float32)
tensor

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.95410156, 8.03125   , 8.03125   , 0.953125  ], dtype=float32)>

In [ ]:
#valores afectados originales en float

In [25]:
valores_afectados=tf.convert_to_tensor([0.953125, 0.953125, 0.948242188, 0.953125])
valores_afectados

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.953125 , 0.953125 , 0.9482422, 0.953125 ], dtype=float32)>

In [26]:
inyec_convert_float(valores_afectados,faults)

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.95410156, 8.984375  , 8.979492  , 0.953125  ], dtype=float32)>

In [40]:
valores_afectados= [0.953125,  0.953125,   0.948242188,  0.953125]
nuevos_valores   = [ 0.95410156, 8.03125   , 8.03125   , 0.953125]
vals_afect_error=  [0.95410156, 8.984375  , 8.979492  , 0.953125]

# En este ejemplo lo que veo que la diferencia entre el valor con la tecnica de redondeo
# y el valor que se insertaria con el error sin tecnica no difieren en gran medida
#   

In [ ]:
## Diferencia entre lo que se cologa en el experimento original y los valores con el Round

In [42]:
tf.subtract(vals_afect_error,tensor)

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.       , 0.953125 , 0.9482422, 0.       ], dtype=float32)>

In [ ]:
x0xxxxxxx1xxxxxx', 'x1xxxxxxx0xxxxxx', 'x1xxxxxxx1xxxxxx', 'x1xxxxxxx0xxxxxx', 'x1xxxxxxx1xxxxxx'

In [ ]:
locs= locs[11:16]  
error_mask=error_mask[11:16]


valores afectados: 1960 1960 1944 1944 1944
tensor_with_error :2024 18344 18392 18328 18392
valores_a cambiar :True  True  True False
    
v_m:                2024 18344 18392 18328 18392
v_b :               1960 1960 1944 1944 1944
mask                64 16384 16384 16384 16384
newValues ANTES     1920    0    0    0    0
newValues DESPUES   1920    0    0    0    0
tensor_act         (1920    0    0    0    0] 
tensor roun luego de inyectar errores entero [1984 16384 16448 16384 16448]                    
affectedValues                           [0.95703125 0.95703125 0.94921875 0.94921875 0.94921875]
newValues para actualizar el tensor final [0.96875 8 8.03125 8 8.03125]    
#test #1
                   
0b0000011111101000 v_m 2024
0b0000011110101000 v_b 1960
0b0000000001000000  mask 64
0b0000000000111111  mask-1 63 
0b1111111111000000  n_mask 65472
0b0000011110101000 v_b 1960  
0b0000011110000000 1920 nuev_val   

#test #2 
                      
0b0100011110101000 v_m  18344 
0b0000011110101000 v_b 1960                    
0b0100000000000000 mask 16384
0b1011111111111111 n_mask 
0b0011111111111111  mask-1 16383                  
0b1100000000000000
0b0000011110101000 v_b 1960 and
0b0000000000000000 nuev_val 0
                    
                    
test# 3   
0b0100011111011000 v_m 18392                   
0b0000011110011000 v_b 1944
0b0100000000000000 mask 16384
0b0011111111111111  mask-1 16383 
0b1100000000000000 n_mask
0b0000011110011000 v_b 1944 and
0b0000000000000000 nuev_val 0  
 
                    
test#4  0
0b100011110011000 v_m 18328                   
0b000011110011000 v_b 1944
0b010000000000000 mask 16384  
0b011111111111111 mask-1  
0b100000000000000 n_mask                    
0b000011110011000 v_b 1944 and                   
0b000000000000000  nuev_val 0 
                    
test#5  0    Es igual al test#3                
                    

In [32]:
bin(16383)

'0b11111111111111'

In [29]:
int('0b1011111111111111',2)

49151

In [34]:
new_Values=tf.convert_to_tensor([1920 ,   0,    0 ,   0 ,   0])
mod_list= ([49151, 64], [65471, 16384], [65535, 16448], [65471, 16384], [65535, 16448])
faults=tf.convert_to_tensor(mod_list)
new_Values

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([1920,    0,    0,    0,    0])>

In [38]:
tensor_round=inyectar_error(new_Values,faults)
tensor_round

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([ 1984, 16384, 16448, 16384, 16448])>

In [39]:
tensor = tf.where(tf.greater_equal(tensor_round,shift), shift-tensor_round , tensor_round )
tensor = tf.cast(tensor/factor,dtype = tf.float32)
tensor

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0.96875, 8.     , 8.03125, 8.     , 8.03125], dtype=float32)>

In [ ]:
#valores afectados originales en float

In [40]:
valores_afectados=tf.convert_to_tensor([0.95703125, 0.95703125, 0.94921875, 0.94921875, 0.94921875])
valores_afectados

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.95703125, 0.95703125, 0.94921875, 0.94921875, 0.94921875],
      dtype=float32)>

In [44]:
vals_afect_error=inyec_convert_float(valores_afectados,faults)
vals_afect_error

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.98828125, 8.957031  , 8.980469  , 8.949219  , 8.980469  ],
      dtype=float32)>

In [ ]:
valores_afectados= [0.95703125, 0.95703125,  0.94921875, 0.94921875, 0.94921875]
nuevos_valores   = [0.96875,    8.     ,     8.03125,     8.     ,   8.03125]
vals_afect_error=  [0.98828125, 8.957031  ,  8.980469  ,  8.949219  , 8.980469  ] 

In [ ]:
index_list [[0, 28, 28, 1], [0, 29, 41, 1], [0, 30, 10, 1], [0, 31, 7, 1], [0, 32, 4, 1]]

In [ ]:
## Diferencia entre lo que se cologa en el experimento original y los valores con el Round

In [45]:
tf.subtract(vals_afect_error,tensor)

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.01953125, 0.95703125, 0.94921875, 0.94921875, 0.94921875],
      dtype=float32)>